# Double Funded Decentralized

Open a double-funded channel without the aid of an intermediary being present.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [ ]:
import sys

# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')
sys.path.append('/notebooks/p2oc/p2oc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg, routermsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009', cert_path='/ali-lnd/tls.cert', macaroon_path='/tmp/ali.macaroon')
bob = LndRpc(host='bob-lnd:10009', cert_path='/bob-lnd/tls.cert', macaroon_path='/tmp/bob.macaroon')

## Fund Ali and Bob

In [ ]:
import os
import time
import bitcoin
from btc_rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
ali_address = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
bob_address = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))

try:
    brpc.loadwallet('miner')
except:
    brpc.createwallet('miner')
    
miner_address = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
_ = list(brpc.generatetoaddress(400, miner_address))
_ = list(brpc.generatetoaddress(10, ali_address.address))
_ = list(brpc.generatetoaddress(10, bob_address.address))
_ = list(brpc.generatetoaddress(120, miner_address))

# Give LNDs time to catch up
time.sleep(5)

In [ ]:
print(f"""alice balance={ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}
bob balance={bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}""")

## Alice (Taker)
Creates public offer to request inbound liquidity in exchange for a fee.

In [ ]:
import bitcoin.core as bc

# Taker needs to pay premium to open channel
premium_amount = int(0.001 * bc.COIN) # premium Ali is willing to pay
fund_amount = int(0.16 * bc.COIN) # requested inbound capacity

In [ ]:
from offer import OfferCreator

In [ ]:
ali_offer_creator = OfferCreator(lnd_rpc=ali, btc_rpc=brpc)
ali_offer, ali_inputs, ali_key_desc = ali_offer_creator.create(
    premium_amount=premium_amount,
    fund_amount=fund_amount
)
ali_offer = ali_offer.serialize()
ali_offer

## Bob (Maker)

Accepts offer and sends funding transaction response.

In [ ]:
from offer import OfferValidator, OfferResponse, Offer, FundingTx, ChannelManager, genpubkey

ali_offer = Offer.deserialize(ali_offer)
offer_validator = OfferValidator(lnd_rpc=ali, btc_rpc=brpc)
assert offer_validator.validate(ali_offer)

Allocate funds

In [ ]:
# XXX: Bob pays funding tx fee but he does not have to
bob_offer_creator = OfferCreator(lnd_rpc=bob, btc_rpc=brpc)
bob_offer, bob_inputs, bob_key_desc = bob_offer_creator.create(
    premium_amount=ali_offer.premium_amount,
    fund_amount=ali_offer.fund_amount,
    fund=True
)

Create funding transaction

In [ ]:
funding_tx_creator = FundingTx()
funding_tx, funding_output_idx = funding_tx_creator.create(
    maker_pubkey=bob_offer.chan_pubkey,
    taker_pubkey=ali_offer.chan_pubkey,
    premium_amount=ali_offer.premium_amount,
    fund_amount=ali_offer.fund_amount,
    maker_inputs=bob_offer.inputs,
    taker_inputs=ali_offer.inputs,
    maker_change_output=bob_offer.change_output,
    taker_change_output=ali_offer.change_output,
)

funding_tx

Connect to taker (Ali)

In [ ]:
# TODO: Assumes whoever creates offer is accessible
connect_peer_req = lnmsg.ConnectPeerRequest(
    addr=lnmsg.LightningAddress(
        pubkey=ali_offer.node_pubkey,
        # TODO: Why can't we include the port?
        host=ali_offer.node_host.split(":")[0]
))

try:
    bob.lnd.ConnectPeer(connect_peer_req)
except:
    pass

# Confirm that we are connected
peers = bob.lnd.ListPeers(lnmsg.ListPeersRequest())
assert len(peers.peers) == 1 and peers.peers[0].pub_key == ali_offer.node_pubkey

Register shim to prepare for channel opening

In [ ]:
channel_id = ChannelManager.register_shim(
    lnd_rpc=bob,
    fund_amount=ali_offer.fund_amount,
    premium_amount=ali_offer.premium_amount,
    local_key_desc=bob_key_desc,
    remote_pubkey=ali_offer.chan_pubkey,
    funding_txid=funding_tx.GetTxid(),
    funding_output_idx=funding_output_idx,
)

Sign transaction

In [ ]:
bob_signed_witness = FundingTx.signed_witness(
    funding_inputs=bob_inputs,
    funding_tx=funding_tx,
    # for simplicity Bob's input is 0
    input_idx=0,
    lnd_rpc=bob
)

Send offer response

In [ ]:
offer_response = OfferResponse(
    offer_id=ali_offer.offer_id,
    node_host=bob_offer.node_host,
    node_pubkey=bob_offer.node_pubkey,
    channel_id=channel_id,
    chan_pubkey=bob_key_desc.raw_key_bytes,
    funding_output_idx=funding_output_idx,
    funding_tx=funding_tx,
    signed_witness=bob_signed_witness
)

offer_response = offer_response.serialize()
offer_response

## Ali (Taker)

Checks offer reply and commits funding TX.

In [ ]:
offer_response = OfferResponse.deserialize(offer_response)

Check transaction and funding output

In [ ]:
validator = OfferValidator(lnd_rpc=ali, btc_rpc=brpc)
validator.validate_offer_response(ali_offer, offer_response)

Open pending channel

In [ ]:
ChannelManager.open_pending_channel(
    lnd_rpc=ali,
    key_desc=ali_key_desc,
    offer_response=offer_response,
    premium_amount=ali_offer.premium_amount,
    fund_amount=ali_offer.fund_amount
)

Sign funding tx

In [ ]:
from offer import FundingTx

# Combine signatures
ali_signed_witness = FundingTx.signed_witness(
    funding_inputs=ali_inputs,
    funding_tx=offer_response.funding_tx,
    # Assume we are the last one
    input_idx=len(offer_response.funding_tx.vin) - 1,
    lnd_rpc=ali
)

final_funding_tx = FundingTx.create_signed_funding_tx(offer_response.funding_tx, [
    offer_response.signed_witness,
    ali_signed_witness
])
final_funding_tx

Submit funding transaction

In [ ]:
final_funding_tx_id = brpc.sendrawtransaction(final_funding_tx)
final_funding_tx_id

In [ ]:
assert final_funding_tx_id == offer_response.funding_tx.GetTxid()

In [ ]:
_ = list(brpc.generatetoaddress(6, miner_address))

## Check that the channel has been opened

In [ ]:
# should be no pending channels
assert len(ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest()).pending_open_channels) == 0

ali.lnd.ListChannels(lnmsg.ListChannelsRequest())